In [ ]:
import os
os.getcwd()

'/rdsgpfs/general/project/aandedemand/live/satellite/junin/deforestation_forecasting/python_code/Notebooks'

In [ ]:
class Conv_3D(torch.nn.Module):
    def __init__(self, input_dim=(2,8),
                 hidden_dim=(16,32,32),
                 kernel_size=((5,5),(2,5,5),(5,5)),
                 levels=(10,),
                 dropout = 0.2):
        super(Conv_3D, self).__init__()

        self.levels = levels
        self.hidden_dim = hidden_dim

        self.conv_2D = torch.nn.Sequential(
            torch.nn.Conv2d(input_dim[0],hidden_dim[0],kernel_size = kernel_size[0]),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(hidden_dim[0]),

            torch.nn.Conv2d(hidden_dim[0],hidden_dim[0],kernel_size = kernel_size[0]),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(hidden_dim[0]))

        self.conv_3D = torch.nn.Sequential(
                        torch.nn.Conv3d(in_channels = input_dim[1],
                                        out_channels = hidden_dim[1],
                                        kernel_size = kernel_size[1]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm3d(hidden_dim[1]),

                        torch.nn.Conv3d(in_channels = hidden_dim[1],
                                        out_channels = hidden_dim[1],
                                        kernel_size = kernel_size[1]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm3d(hidden_dim[1]))

        self.final = torch.nn.Sequential(
                        torch.nn.Conv2d(hidden_dim[0]+hidden_dim[1], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]),

                        torch.nn.Conv2d(hidden_dim[2], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]),

                        torch.nn.Conv2d(hidden_dim[2], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]),

                        torch.nn.Conv2d(hidden_dim[2], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]),

                        torch.nn.Conv2d(hidden_dim[2], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]),

                        torch.nn.Conv2d(hidden_dim[2], hidden_dim[2], kernel_size[2]),
                        torch.nn.ReLU(),
                        torch.nn.BatchNorm2d(hidden_dim[2]))

        ln_in = 0
        for i in levels:
            ln_in += hidden_dim[2]*i*i

        self.ln = torch.nn.Sequential(
            torch.nn.Linear(ln_in,100),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(100),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(100, 1))

        self.sig = torch.nn.Sigmoid()


    def forward(self, data , sigmoid = True ):

        s , x = data

        s = self.conv_2D.forward(s)
        x = self.conv_3D.forward(x)
        x = x.squeeze(dim = 2 )
        x = torch.cat((x,s),dim = 1)
        x = self.final.forward(x)
        x = spp_layer(x, self.levels)
        x= self.ln(x)
        if sigmoid:
            x = self.sig(x)

        return x.flatten()

# Initial parameters:

In [ ]:
#set image parameters
size = 45
#set model parameters for 3D_CNN
input_dim= (2,8)
hidden_dim=(16,32,32)
kernel_size=((5,5),(2,5,5),(5,5))
levels=(13,)
dropout = 0.3

In [ ]:
model = Conv_3D(
    input_dim = input_dim,
    hidden_dim = hidden_dim,
    kernel_size= kernel_size,
    levels=levels,
    dropout = dropout)

In [ ]:
b, c1, c2, t, h, w = 3, 2, 8, 3, size, size
s = torch.rand(2,c1,size,size)
x = torch.rand(2,c2,t,size,size)
data = (s,x)
model(data)

tensor([0.5335, 0.4771], grad_fn=<AsStridedBackward>)

In [ ]:
def check_input_size(size, kernel_size):
    print("Input image spatial size:",size)
    print("Changes of the spatial size in the two branches (2D_cov and 3D_conv)")
    for i in range(2):
        print("\tLayer",i+1)
        print("\tkernel_size: ", kernel_size[0])
        size = np.array(size) - np.array(kernel_size[0]) + 1
        print("\tSize after layer %d is applied:"%(i+1), size)
        print()
    print("Changes of the spatial size in the final brach:")
    for i in range(6):
        print("\tLayer",i+1)
        print("\tkernel_size: ", kernel_size[2])
        size = np.array(size) - np.array(kernel_size[2])  + 1
        print("\tSize after layer %d is applied:"%(i+1), size)
        print()

check_input_size((size,size), kernel_size)

Input image spatial size: (45, 45)
Changes of the spatial size in the two branches (2D_cov and 3D_conv)
	Layer 1
	kernel_size:  (5, 5)
	Size after layer 1 is applied: [41 41]

	Layer 2
	kernel_size:  (5, 5)
	Size after layer 2 is applied: [37 37]

Changes of the spatial size in the final brach:
	Layer 1
	kernel_size:  (5, 5)
	Size after layer 1 is applied: [33 33]

	Layer 2
	kernel_size:  (5, 5)
	Size after layer 2 is applied: [29 29]

	Layer 3
	kernel_size:  (5, 5)
	Size after layer 3 is applied: [25 25]

	Layer 4
	kernel_size:  (5, 5)
	Size after layer 4 is applied: [21 21]

	Layer 5
	kernel_size:  (5, 5)
	Size after layer 5 is applied: [17 17]

	Layer 6
	kernel_size:  (5, 5)
	Size after layer 6 is applied: [13 13]

